# ASL sign challenge exploration

[Model](## Building the model using Keras)

Data includes 3000 images per sign in ASL

In [ ]:
import os
import numpy as np
from PIL import Image
from IPython.display import clear_output
from skimage.transform import resize
%matplotlib inline

# kaggle input data path (change when running a local notebook)
TRAIN_DIR = "../input/asl_alphabet_train/asl_alphabet_train/"

# Sort signs A through Z then del, nothing, space
class_folders = np.sort(os.listdir(TRAIN_DIR))

In [ ]:
''' initialising values for model building. these will be tested with alterations
    to try and optimise the model  '''

# limiting data for prototyping model
cutoff = 500

# validation farction
val_frac = 0.1

# model parameters
batch_size = 100
epochs = 10
filters = 16
dropouts = .25
learning_rate = 0.005
kernel_size = [(3,3), (3,3), (3,3)]

# keras model checkpoint file
run_name = str(epochs)+'epochs_'+str(learning_rate)+'LR.ckpt'

In [ ]:
# Load in data to numpy array, slow on cpu, memory fill?
### Change this from a loop? Could use DataFrame

labels = []
data = []

label_no = 0
for class_name in class_folders:
    clear_output()
    
    label_dir = TRAIN_DIR + class_name + "/"
    class_files = os.listdir(label_dir)
    for i,file in enumerate(class_files):
        
        # print("On class {} and file {}".format(label_no, i))
        img = np.array(Image.open(label_dir + file))
        
        # removing the blue border on all images
        trim_img = img[15:-10,15:-10]
        
        # Downsizing images to 64x64
        data.append(resize(trim_img, (64,64)))
        labels.append(label_no)
        
        # Cutoff samples for each sign to speed testing
        if i > cutoff:
            break
    label_no += 1
    
    
data = np.array(data)
labels = np.array(labels)

In [ ]:
print(np.shape(data))

In [ ]:
# checking what the formatted data looks like
import matplotlib.pyplot as plt
from skimage.transform import resize

fig=plt.figure(figsize=(18, 18), dpi= 80, facecolor='w', edgecolor='k')
rows = 5
columns = 6
for i in range(1,30):
    fig.add_subplot(rows, columns, i)
    plt.imshow(resize(data[i*cutoff,:,:,:], (64,64)))
    

## Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

Might be low res but will run better for a first passthrough. Blue borders are removed.

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

# Take a portion of data for validation
X_train, X_val, Y_train, Y_val = train_test_split(data, 
                          labels, test_size=0.1, random_state=4)

Y_train = np_utils.to_categorical(Y_train)
Y_val = np_utils.to_categorical(Y_val)

## Building the model using Keras

### Network

The network consists of two main components :

#### Convolutional layers : 

the convolutional layer is responsible for the convolutional operation in which feature maps identifies features in the images. and is usually followed by two types of layers which are :

   Dropout : Dropout is a regulization technique where you turn off part of the network's layers randomally to increase regulization and hense decrease overfitting. We use when the training set accuracy is muuch higher than the test set accuracy.
    Max Pooling : The maximum output in a rectangular neighbourhood. It is used to make the network more flexible to slight changes and decrease the network computationl expenses by extracting the group of pixels that are highly contributing to each feature in the feature maps in the layer.

#### Dense layers : 

The dense layer is a fully connected layer that comes after the convolutional layers and they give us the output vector of the Network.
    
[Number of parameteres for convolution layers?](https://datascience.stackexchange.com/questions/17064/number-of-parameters-for-convolution-layers)

### Compilation
Before training a model, you need to configure the learning process, which is done via the  `compile` method, receiving 3 arguments: `optimizer`, `loss function`,  `list of metrics`. 

[Keras Docs: Compilation](https://keras.io/getting-started/sequential-model-guide/#compilation)

[Overview of gradient descent algorithms](http://ruder.io/optimizing-gradient-descent/)

Adam is a method for Stochastic Optimization. See the following papers: 

[Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)

[On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)

Cost function : 
It is a measure of the overall loss in our network after assigning values to the parameters during the forward phase so it indicates how well the parameters were chosen during the forward probagation phase.

Optimizer : 
It is the gradiant descent algorithm that is used. We use it to minimize the cost function to approach the minimum point. We are using adam optimizer which is one of the best gradient descent algorithms.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# Build a model using 2D Convolution
def training(weights_path):
    model = Sequential()
    
    #layers
    model.add(Conv2D(filters = filters, kernel_size = kernel_size[0], padding = 'Same', 
                    activation = 'relu', input_shape = (64,64,3)))
    model.add(Conv2D(filters = filters, kernel_size = kernel_size[0], padding = 'Same', 
                    activation = 'relu'))

    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 2*filters, kernel_size = kernel_size[1], padding = 'Same',
                    activation = 'relu'))
    model.add(Conv2D(filters = 2*filters, kernel_size = kernel_size[1], padding = 'Same',
                    activation = 'relu'))
    
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 4*filters, kernel_size = kernel_size[2], padding = 'Same',
                    activation = 'relu'))
    model.add(Conv2D(filters = 4*filters, kernel_size = kernel_size[2], padding = 'Same',
                    activation = 'relu'))
    
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(dropouts))
    
    model.add(Flatten())
    model.add(Dense(8*filters, activation = 'relu'))
    model.add(Dropout(dropouts))
    model.add(Dense(label_no, activation = 'softmax'))
    
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', 
                    metrics = ['accuracy'])
    
    #if os.path.isfile(weights_path):
    #    model.load_weights(weights_path)
    
    return model

model = training(None)
model.summary()

In [ ]:
saver = ModelCheckpoint(run_name, monitor="val_loss")

history = model.fit(X_train, Y_train, batch_size = batch_size, 
                    epochs = epochs, validation_data = (X_val, Y_val),
                    verbose = 2, callbacks = [saver])

#history = model.fit_generator()

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
fig=plt.figure(figsize=(15, 15))
# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(29))
plt.savefig(run_name[:-5]+'_conf.png')

In [ ]:
test42 = model.predict(X_val[10:11,:,:,:])
#plt.plot(np.arange(label_no),test42)
#plt.figure()
#plt.imshow(X_val[10,:,:,:])
# finding the error, are our X_vals still good? = True
(Y_val[10])

In [ ]:
plt.figure()
plt.imshow(X_val[10,:,:,:])

In [ ]:
import seaborn as sns
y_val = [np.argmax(one_hot) for one_hot in Y_val]
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.distplot(y_val, kde=False, rug=True, bins=29)


In [ ]:
history.history.keys()

In [ ]:
plt.figure()
for keys in ['val_loss', 'loss']:
    plt.plot(np.arange(0,10), history.history[keys], label=keys)
plt.legend()

plt.figure()
for keys in ['val_acc', 'acc']:
    plt.plot(np.arange(0,10), history.history[keys], label=keys)
plt.legend()
plt.title(max(history.history['val_acc']))
plt.savefig(run_name[:-5]+'_acc.png')

In [ ]:
history.history['val_acc']

# View false positives

Lets have a look at where the network hasn't performed well, it may point towards flaws in the method/model architecture or flaws with the data such as poor lighting. 



In [ ]:
# Predict all images n the training set
train_pred = model.predict(X_train)


# Analyse Results

Lets have a look at the model training curves to see if we have set the hyperparameters corectly

In [ ]:
from keras import metrics